In [1]:
import pandas as pd
import numpy as np
import mozart_piano_sonatas.utils.feature_matrices as fm
from utils import notetype
import glob
import sys
import os.path as path
import tqdm
from importlib import reload

In [2]:
chord_types = {
    'M': [0,1,4],
    'm': [0,1,-3],
    'o': [0,-3,-6],
    '+': [0,4,8],
    'Mm7': [0,1,4,-2],
    'mm7': [0,1,-3,-2],
    'MM7': [0,1,4,5],
    'mM7': [0,1,-3,5],
    '%7': [0,-3,-6,-2],
    'o7': [0,-3,-6,-9],
    '+7': [0,4,8,-2]
}

In [3]:
corpus = "dcml_corpora/annotations/Bach-Suites/"
piece = 'BWV812_01_Allemande'

In [4]:
def total_onsets(events, measures):
    moffsets = measures.act_dur.values.cumsum()
    monsets = moffsets - measures.act_dur.values
    mshifts = measures.offset.values
    mi = events.mc - 1
    return events['onset'] + monsets[mi] - mshifts[mi]

In [5]:
def merge_ties(notes):
    notes = notes.copy()
    beginnings = notes[notes.tied == 1]
    continues = notes[notes.tied < 1]
    for i in beginnings.index:
        on = notes.total_onset[i]
        off = notes.total_offset[i]
        midi = notes.midi[i]
        tpc = notes.tpc[i]
        while True:
            cont = continues[(continues.total_onset == off) &
                             (continues.midi == midi) &
                             (continues.tpc == tpc)].first_valid_index()
            if cont is None:
                break
            off = continues.total_offset[cont]
            if continues.tied[cont] == -1:
                break
        notes.at[i, 'total_offset'] = off
        notes.at[i, 'duration'] = off - on
    return notes[~(notes.tied < 1).fillna(False)]

In [6]:
def load_dfs(corpus, piece):
    measures = fm.read_tsvs(corpus + 'measures', [piece])
    
    notes = fm.read_tsvs(corpus + 'notes', [piece])
    notes['total_onset'] = total_onsets(notes, measures)
    notes['total_offset'] = notes.total_onset.values + notes.duration.values
    notes = merge_ties(notes)
    max_offset = notes.total_offset.values.max()
    
    harmonies = fm.read_tsvs(corpus + 'harmonies', [piece])
    harmonies['total_onset'] = total_onsets(harmonies, measures)
    harmonies['total_offset'] = np.append(harmonies.total_onset.values[1:], max_offset)
    
    return notes, harmonies

In [7]:
def get_chords(notes, harmonies, id_offset=0):
    key = fm.name2tpc(harmonies.globalkey[0])
    
    chordids = np.empty(0, dtype=int)
    labels   = np.empty(0, dtype=str)
    fifths   = np.empty(0, dtype=int)
    types    = np.empty(0, dtype=str)
    highest_id = id_offset
    
    for i, ih in enumerate(harmonies.index):
        if harmonies.chord[ih] == '@none':
            continue
        on = harmonies.total_onset[ih]
        off = harmonies.total_offset[ih]
        label = harmonies.chord_type[ih]
        root = harmonies.root[ih] + key
        
        inotes = (notes.total_offset > on) & (notes.total_onset < off)
        pitches = notes.tpc[inotes].values - root
        chord_tones = chord_types[label]
        note_types = [notetype(p, pitches, chord_tones) for p in pitches]
        chordids = np.append(chordids, np.repeat(i + id_offset, len(pitches)))
        labels   = np.append(labels, np.repeat(label, len(pitches)))
        fifths   = np.append(fifths, pitches)
        types    = np.append(types, note_types)
        highest_id = i + id_offset
        
    chords_df = pd.DataFrame({
        'chordid': chordids,
        'label': labels,
        'fifth': fifths,
        'type': types})
    return chords_df, highest_id

In [8]:
def get_chords_from_piece(folder, file, id_offset=0):
    notes, harmonies = load_dfs(folder, file)
    return get_chords(notes, harmonies, id_offset)

In [9]:
chords, max_id = get_chords_from_piece(corpus, piece, 12)
print(f"{max_id=}")
chords

max_id=169


,chordid,label,fifth,type
0,12,m,0,chordtone
1,12,m,0,chordtone
2,12,m,0,chordtone
3,12,m,1,chordtone
4,13,m,1,chordtone
...,...,...,...,...
719,169,M,0,chordtone
720,169,M,4,chordtone
721,169,M,1,chordtone
722,169,M,0,chordtone


In [19]:
def get_chords_from_files(filelist):
    offset = 0
    all_chords = None
    for folder, file in tqdm.tqdm(filelist):
        try:
            chords, max_id = get_chords_from_piece(folder, file, offset)
            all_chords = chords if (all_chords is None) else all_chords.append(chords)
            offset = max_id + 1
        except FileNotFoundError:
            print(f'file not found for {folder} {file}')
            continue
        except ValueError:
            print(f'ValueError in {folder} {file}')
            continue
        except:
            print(f'failed for unknown reasons: {folder} {file}')
            #raise Exception(f"failed file: {folder} {file}")
    return all_chords.reset_index(drop=True)

In [20]:
def get_corpus_pieces(corpus):
    dirs = glob.glob(corpus + 'annotations/*/')
    files = [(d, path.splitext(path.basename(f))[0])
             for d in dirs
             for f in glob.glob(d + 'notes/*.tsv')]
    return files

In [21]:
pieces = get_corpus_pieces('dcml_corpora/')

In [22]:
all_chords = get_chords_from_files(pieces)
all_chords

 14%|█▍        | 121/845 [08:11<10:48,  1.12it/s]  

file not found for dcml_corpora/annotations/Couperin-gouts_reunis/ c13n01_unisson


 15%|█▌        | 127/845 [08:13<05:51,  2.04it/s]

file not found for dcml_corpora/annotations/Couperin-gouts_reunis/ c13n02_air


 16%|█▌        | 131/845 [08:15<03:50,  3.09it/s]

...

failed for unknown reasons: dcml_corpora/annotations/Beethoven-Sonatas/ 10-3


 97%|█████████▋| 820/845 [22:15<00:52,  2.11s/it]

failed for unknown reasons: dcml_corpora/annotations/WFBach-Sonatas/ Bach_WF_F03_Sonate_D-Dur_03


100%|██████████| 845/845 [22:34<00:00,  1.60s/it]


,chordid,label,fifth,type
0,0,m,0,chordtone
1,0,m,0,chordtone
2,0,m,0,chordtone
3,1,M,4,chordtone
4,2,m,-3,chordtone
...,...,...,...,...
879153,137902,M,0,chordtone
879154,137903,Mm7,4,chordtone
879155,137903,Mm7,0,unknown
879156,137903,Mm7,0,unknown


In [23]:
get_chords_from_piece('dcml_corpora/annotations/Chopin-Mazurkas/', 'BI18op68-2')

AttributeError: 'DataFrame' object has no attribute 'globalkey'

In [24]:
all_chords.to_csv('data/dcml_harmonies.tsv', sep='\t', index=False)

In [87]:
notes_dbg = fm.read_tsvs('dcml_corpora/annotations/Schumann-Liederkreis_op39/notes/', ['op39n08'])
measures_dbg = fm.read_tsvs('dcml_corpora/annotations/Schumann-Liederkreis_op39/measures/', ['op39n08'])
total_onsets(notes_dbg, measures_dbg)

filename  _id
op39n08   0      1/16
          1      1/16
          2       1/8
          3       1/8
          4       1/8
                 ... 
          757      19
          758      19
          759      19
          760      19
          761      19
Name: onset, Length: 762, dtype: object

In [75]:
df_dbg.tied

filename  _id
op39n08   0      <NA>
          1      <NA>
          2      <NA>
          3      <NA>
          4      <NA>
                 ... 
          757    <NA>
          758    <NA>
          759      -1
          760    <NA>
          761      -1
Name: tied, Length: 762, dtype: Int64